In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### <b>문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)</b>
<b>문제 설명</b><br>
: MessagesState를 사용하여 고객의 카페 관련 문의에 자동으로 응답하는 시스템을 만드세요. 고객이 메뉴, 가격, 추천 등에 대해 질문하면 카페 메뉴 데이터를 바탕으로 적절한 답변을 생성하는 시스템을 구현하세요.

<b>MessagesState</b>: 메시지 리스트를 자동으로 관리하는 LangGraph의 특별한 상태 클래스<br>
<b>HumanMessage/AIMessage</b>: 사용자와 AI의 메시지를 구분하는 LangChain의 메시지 클래스<br>
<b>자연어 처리</b>: 사용자의 텍스트 입력을 분석하여 의도를 파악하는 과정<br>
<b>상태 확장</b>: MessagesState를 상속받아 추가 필드를 포함하는 방법<br>

<b>요구사항</b>
- MessagesState 사용
- 질문 유형 분류 (메뉴 문의, 가격 문의, 추천 요청)
- 각 유형별 맞춤 응답 생성
- 대화 이력 유지


In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.documents import Document
from typing import List

# MessagesState 상속 → 대화 이력 + 검색 결과 저장
class CafeMessagesState(MessagesState):
    search_docs: List[Document]

# Rag Chain 구성
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Embedding 모델 로드
embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# menu_db 벡터 저장소 로드 → 의미론적 검색용
menu_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# 의미론적 검색 함수
def search_menu(user_message: str, k: int = 3):
    return menu_db.similarity_search(user_message, k=k)

# 고급 정보 추출 함수
import re

def extract_menu_info(doc: Document) -> dict:
    price_match = re.search(r'₩([\d,]+)', doc.page_content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', doc.page_content)
    
    return {
        "name": doc.metadata.get("menu_name", "Unknown"),
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# 문의 유형 분류
def classify_query(user_message: str) -> str:
    if "가격" in user_message:
        return "price"
    elif "추천" in user_message:
        return "recommend"
    else:
        return "menu"

# 카페 상담 노드
def cafe_agent_node(state: CafeMessagesState) -> CafeMessagesState:
    user_message = state["messages"][-1].content
    query_type = classify_query(user_message)

    print(f"[Agent] Query type: {query_type}")

    # 유형별 검색 전략
    if query_type == "price":
        docs = menu_db.similarity_search(user_message, k=5)
    elif query_type == "recommend":
        docs = menu_db.similarity_search(user_message, k=3)
        if not docs:
            docs = menu_db.similarity_search("인기 메뉴", k=3)
    else:
        docs = search_menu(user_message, k=4)

    # 정보 저장
    state["search_docs"] = docs

    # 응답 생성
    if not docs:
        reply = AIMessage(content="죄송합니다. 해당 메뉴를 찾을 수 없습니다.")
    else:
        infos = [extract_menu_info(doc) for doc in docs]
        reply_text = "\n\n".join(
            f"{info['name']} - {info['price']}\n{info['description']}" for info in infos
        )
        reply = AIMessage(content=reply_text)

    # 대화 이력 유지
    state["messages"].append(reply)
    return state

# 그래프 구성
from langgraph.graph import StateGraph, START, END

builder = StateGraph(CafeMessagesState)
builder.add_node("cafe_agent_node", cafe_agent_node)
builder.add_edge(START, "cafe_agent_node")
builder.add_edge("cafe_agent_node", END)

graph = builder.compile()

# 실행 예시
init_state = CafeMessagesState(
    messages=[HumanMessage(content="아이스 아메리카노 가격 알려줘")],
    search_docs=[]
)

final_state = graph.invoke(init_state)

# 결과 출력
for msg in final_state["messages"]:
    role = "User" if isinstance(msg, HumanMessage) else "AI"
    print(f"[{role}] {msg.content}")



[Agent] Query type: price
[User] 아이스 아메리카노 가격 알려줘
[AI] 아이스 아메리카노 - ₩4,500
진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.

아메리카노 - ₩4,500
진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

프라푸치노 - ₩7,000
에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다.

카푸치노 - ₩5,000
에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.
